# Introduction to PyPSA

**Python for Power System Analysis (PyPSA)**

PyPSA is a python package that supports energy and electricity system modeling. The PyPSA package provides a framework for simulating and optimizing energy and electricity systems. PyPSA provides users structured components for which they can store the data which populates an optimization model.

## Learning Objectives

By the end of this session, you will:
1. Understand what PyPSA is and its core capabilities
2. Learn the basic components of a power system model in PyPSA
3. Build your first simple power system network
4. Run an economic dispatch model
5. Run a basic capacity expansion model
5. Visualize network topology and results

In [ ]:
# Import necessary libraries
import pypsa
import os
import matplotlib.pyplot as plt
import logging

from helpers import (
    plot_generator_marginal_costs,
    plot_energy_balance,
    plot_capacity_comparison
)

logger = logging.getLogger("gurobipy")
logger.propagate = False

pypsa.__version__

## 1. Anatomy of a PyPSA Network

Lets start with an example network. This is a pypsa-usa network created for ERCOT with:
- demand data for the year 2030 from the NREL EFS
- 7 buses nodes
- 380 resource regions
- 72 transmission links

In [ ]:
path = os.path.join("../data/examples", "elec_s380_c7a_ec_lv1.5_RPS-REM-TCT-1h_E.nc")
network = pypsa.Network(path)
network

In [ ]:
network.buses

In [ ]:
network.consistency_check()

In [ ]:
print(set(network.generators.carrier))
network.generators

In [ ]:
network.links

In [ ]:
network.get_switchable_as_dense("Generator", "marginal_cost")

In [ ]:
network.lines

In [ ]:
network.storage_units

In [ ]:
network.stores

In [ ]:
network.component_attrs['Generator']

In [ ]:
network.loads

But where is the load data?
The load data lives in the time-series component of loads, called loads_t

In [ ]:
network.loads_t.p_set

Similarly, we have time-series data from generators in the generators_t dictionary. Within the generators_t dictionary there are multiple dataframes with different types of data!

In [ ]:
print("Time-series data keys:", network.generators_t.keys())

In [ ]:
network.snapshots

In [ ]:
network.snapshot_weightings

## Single node economic dispatch analytical example

$$
\min_{p_1, p_2}  {3} p_{1} + 6p_{2}
$$

$$
\text{st:                     } \quad \quad \quad
$$

$$p_{1} \le 4 $$
$$p_{2} \le 6 $$ 
$$p_{1} + p_{2} = 8 $$
$$p_{1}, p_{2} \ge 0 $$



In [ ]:
# Sequential solution reveal for the economic dispatch plot (ADVANCE BY BUTTON OR "ENTER")

import ipywidgets as widgets
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
import numpy as np

def sequential_reveal_economic_dispatch_interactive():
    steps = []

    # Step 0 setup (constructor pattern: each step gets a *fresh* fig,ax)
    def step1():
        fig, ax = plt.subplots(figsize=(6,6))
        p1 = np.linspace(0, 6, 300)
        p2 = np.linspace(0, 8, 300)
        x_line = np.linspace(0, 8, 100)
        p1_feas = np.linspace(2, 4, 100)
        p2_feas = 8 - p1_feas

        ax.set_xlim(0, 6)
        ax.set_ylim(0, 8)
        ax.set_xlabel("$p_1$")
        ax.set_ylabel("$p_2$")
        ax.set_title("Single Node Economic Dispatch Feasible Region")
        plt.show()
    steps.append(step1)

    def step2():
        fig, ax = plt.subplots(figsize=(6,6))
        p1 = np.linspace(0, 6, 300)
        p2 = np.linspace(0, 8, 300)
        x_line = np.linspace(0, 8, 100)
        p1_feas = np.linspace(2, 4, 100)
        p2_feas = 8 - p1_feas

        ax.set_xlim(0, 6)
        ax.set_ylim(0, 8)
        ax.set_xlabel("$p_1$")
        ax.set_ylabel("$p_2$")
        ax.set_title("Single Node Economic Dispatch Feasible Region")
        patch1 = ax.fill_betweenx(p2, 4, 6, color='gray', alpha=0.3, label='p1 ≤ 4')
        ax.legend(handles=[ patch1], loc="upper right")
        plt.show()
    steps.append(step2)

    def step3():
        fig, ax = plt.subplots(figsize=(6,6))
        p1 = np.linspace(0, 6, 300)
        p2 = np.linspace(0, 8, 300)
        x_line = np.linspace(0, 8, 100)
        p1_feas = np.linspace(2, 4, 100)
        p2_feas = 8 - p1_feas

        ax.set_xlim(0, 6)
        ax.set_ylim(0, 8)
        ax.set_xlabel("$p_1$")
        ax.set_ylabel("$p_2$")
        ax.set_title("Single Node Economic Dispatch Feasible Region")
        # line_feas, = ax.plot(p1_feas, p2_feas, color="green", linestyle='--', linewidth=2, label='Feasible boundary')
        patch1 = ax.fill_betweenx(p2, 4, 6, color='gray', alpha=0.3, label='p1 ≤ 4')
        patch2 = ax.fill_between(p1, 6, 8, color='gray', alpha=0.3, label='p2 ≤ 6')
        ax.legend(handles=[ patch1, patch2], loc="upper right")
        plt.show()
    steps.append(step3)

    def step4():
        fig, ax = plt.subplots(figsize=(6,6))
        p1 = np.linspace(0, 6, 300)
        p2 = np.linspace(0, 8, 300)
        x_line = np.linspace(0, 8, 100)
        p1_feas = np.linspace(2, 4, 100)
        p2_feas = 8 - p1_feas

        ax.set_xlim(0, 6)
        ax.set_ylim(0, 8)
        ax.set_xlabel("$p_1$")
        ax.set_ylabel("$p_2$")
        ax.set_title("Single Node Economic Dispatch Feasible Region")
        line_feas, = ax.plot(p1_feas, p2_feas, color="green", linestyle='--', linewidth=2, label='Feasible boundary')
        patch1 = ax.fill_betweenx(p2, 4, 6, color='gray', alpha=0.3, label='p1 ≤ 4')
        patch2 = ax.fill_between(p1, 6, 8, color='gray', alpha=0.3, label='p2 ≤ 6')
        vline = ax.axvline(4, color="k", linestyle="--", lw=1)
        hline = ax.axhline(6, color="k", linestyle="--", lw=1)
        consline, = ax.plot(x_line, 8-x_line, color="k", linestyle="--", lw=1)
        ax.legend(handles=[line_feas, patch1, patch2], loc="upper right")
        plt.show()
    steps.append(step4)

    def step5():
        fig, ax = plt.subplots(figsize=(6,6))
        p1 = np.linspace(0, 6, 300)
        p2 = np.linspace(0, 8, 300)
        x_line = np.linspace(0, 8, 100)
        p1_feas = np.linspace(2, 4, 100)
        p2_feas = 8 - p1_feas

        ax.set_xlim(0, 6)
        ax.set_ylim(0, 8)
        ax.set_xlabel("$p_1$")
        ax.set_ylabel("$p_2$")
        ax.set_title("Single Node Economic Dispatch Feasible Region")
        line_feas, = ax.plot(p1_feas, p2_feas, color="green", linestyle='--', linewidth=2, label='Feasible boundary')
        patch1 = ax.fill_betweenx(p2, 4, 6, color='gray', alpha=0.3, label='p1 ≤ 4')
        patch2 = ax.fill_between(p1, 6, 8, color='gray', alpha=0.3, label='p2 ≤ 6')
        vline = ax.axvline(4, color="k", linestyle="--", lw=1)
        hline = ax.axhline(6, color="k", linestyle="--", lw=1)
        consline, = ax.plot(x_line, 8-x_line, color="k", linestyle="--", lw=1)
        optpoint, = ax.plot(4, 4, 'ro', label="Optimum (4, 4)")
        ax.legend(handles=[line_feas, patch1, patch2, optpoint], loc="upper right")
        plt.show()
    steps.append(step5)

    stepper = widgets.IntSlider(min=1, max=len(steps), step=1, value=1, description="Step", continuous_update=False)
    out = widgets.Output()

    def show_step(change):
        with out:
            clear_output(wait=True)
            steps[stepper.value - 1]()

    stepper.observe(show_step, names="value")

    display(widgets.HTML(""))
    display(stepper)
    display(out)
    show_step(None)  # show first step

sequential_reveal_economic_dispatch_interactive()



In [ ]:
n_simple = pypsa.Network()
n_simple.add("Bus", "bus1")
n_simple.add("Generator", "gen1", bus="bus1", carrier="cheaper_resource", p_nom=4, marginal_cost=3)
n_simple.add("Generator", "gen2", bus="bus1", carrier="expensive_resource", p_nom=6, marginal_cost=6)
n_simple.add("Load", "load1", bus="bus1", p_set=8)
n_simple.add("Carrier", "cheaper_resource", co2_emissions=2, color="green")
n_simple.add("Carrier", "expensive_resource", co2_emissions=10, color="blue")
n_simple.set_snapshots(range(1))
n_simple.optimize(solver_name="gurobi")
n_simple.generators_t.p

## Economic Dispatch with a PyPSA-USA Network

$$
\min_{p} \sum o_{g} p_{g}
$$

$$
\text{st:          } \quad \quad 
$$

$$
p_{g} \le \widetilde{p}^{\max}_{g}\,\overline{P}_{g}
$$

$$
\sum_{g \in G} p_{g} = D
$$


PyPSA's optimization function `pypsa.network.optimize()` builds the optimization model, either a simulation or production cost model depending on the extendable settings of each component.

We need to ensure all components are set to non-extendable before solving the network.

In [ ]:
# You can use this to create an interactive list of components
network.components.keys()

In [ ]:
# Track and set all extendable attributes to False for all components in the network
extendable_attrs_backup = {}

for component in network.components.keys():
    extendable_attrs_backup[component] = {}
    for attr in ["p_nom_extendable", "s_nom_extendable", "e_nom_extendable"]:
        if attr in network.df(component).columns:
            # Backup the current state of the attribute
            extendable_attrs_backup[component][attr] = network.df(component)[attr].copy()
            # Set the attribute to False
            network.df(component)[attr] = False

Remember our load is for 2030, so lets reduce the system load for the sake of this simulation feasibility

In [ ]:
network.loads_t.p_set *= 0.75

In [ ]:
network.snapshots

In [ ]:
plot_generator_marginal_costs(network)

In [ ]:
network.loads_t.p_set.iloc[:72].plot()

In [ ]:
network.generators_t.p_max_pu.loc[network.snapshots[0:7*24]].T.groupby(network.generators.carrier).mean().T.iloc[:72].plot()

In [ ]:
# LCOE = (capital_cost + opex) / (p_nom * capacity_factor * 8760)
lcoes = (network.generators.capital_cost) / (network.generators_t.p_max_pu.mean(axis=0) * 8760)
network.generators['lcoe'] = lcoes

carriers = ["onwind", "solar"]
carrier_labels = {"onwind": "Onshore Wind", "solar": "Solar", "offwind": "Offshore Wind"}
carrier_colors = {"onwind": "tab:blue", "solar": "tab:orange", "offwind": "tab:green"}

plt.figure(figsize=(8, 5))

for carrier in carriers:
    gens = network.generators[
        (network.generators.carrier == carrier) & (network.generators.p_nom_extendable)
    ].copy()
    gens = gens.sort_values("lcoe")
    cumulative = gens["p_nom_max"].cumsum() / 1e3
    plt.step(
        cumulative,
        gens["lcoe"],
        where="post",
        label=carrier_labels[carrier],
        color=carrier_colors[carrier],
        linewidth=2,
    )

plt.xlabel("Cumulative p_nom_max (GW)", fontsize=14)
plt.ylabel("LCOE ($/MWh)", fontsize=14)
plt.title("Renewable Supply Curves", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
network.optimize.create_model(
    snapshots=network.snapshots[0:1],
    multi_investment_periods=True,
)

In [ ]:
network.optimize.solve_model(solver_name="gurobi")
network.optimize.assign_solution()

In [ ]:
# The above sequence of commands is equivalent to:
network.optimize(
    snapshots=network.snapshots[0:1],
    solver_name='gurobi'
)

In [ ]:
# lets look at the results
network.generators_t.p.loc[network.snapshots[0]].groupby(network.generators.carrier).sum().plot(kind="bar")
# only the first time step is solved!

 Great! Most market simulations aren't limited to a single time-step. 
 Let's solve a week of the economic dispatch problem. With multiple time-steps, 
 new constraints are introduced into the formulation:
 - Generator Ramping Constraint
 - Storage Unit State-of-Charge Limit
 - Storage Unit Charge and Discharge Limits
 - Storage Unit Energy Evolution Constraint

 and (if you weren't paying attention) we actually had transmission lines in the last problem! so lets add line flow constraints

$$
\begin{aligned}
\min_{p,\;s^{in},\;s^{out},\;SOC} \quad 
& \sum_{t \in T} \sum_{g \in G} o_{g,t}\, p_{g,t} \\[0.5em]
\text{subject to:} \quad
& p_{g,t} \le \widetilde{p}^{\max}_{g,t}\,\overline{P}_{g} 
&& \forall g \in G,\; t \in T \\[0.5em]
& RD_g \le p_{g,t} - p_{g,t-1} \le RU_g 
&& \forall g \in G,\; t \in T \\[0.5em]
& 0 \le SOC_{s,t} \le h_s\,\overline{S}_{s} 
&& \forall s \in S,\; t \in T \\[0.5em]
& 0 \le s^{out}_{s,t} \le \overline{S}_{s} 
&& \forall s \in S,\; t \in T \\[0.5em]
& 0 \le s^{in}_{s,t} \le \overline{S}_{s} 
&& \forall s \in S,\; t \in T \\[0.5em]
& SOC_{s,t} = SOC_{s,t-1} + \eta^{in}\, s^{in}_{s,t} - \frac{1}{\eta^{out}}\, s^{out}_{s,t} 
&& \forall s \in S,\; t \in T \\[0.5em]
& \widetilde{f}_{b,t}^{\text{min}} \bar{f}_{b} 
  \leq f_{b,t} 
  \leq \widetilde{f}_{b,t}^{\text{max}} \bar{f}_{b} 
  && \forall b \in B,\; t \in T_a     \\[0.5em]
& \sum_{g \in G} p_{g,t,n}
  + \sum_{s \in S} s^{out}_{s,t,n}
  - \sum_{s \in S} s^{in}_{s,t,n}
  = D_{t,n}
&& \forall n \in N,\; t \in T
\end{aligned}
$$


In [ ]:
network.optimize(
    snapshots=network.snapshots[0:7*24],
    solver_name='gurobi'
)

In [ ]:
network.generators_t.p.loc[network.snapshots[0:7*24]].T.groupby(network.generators.carrier).sum()

But this only includes generators! we have storage units in out model and potentially stores, links, etc in other models!

In [ ]:
network.carriers.loc['AC_exp', 'color'] = "#000000"

In [ ]:
plot_energy_balance(network, 7*24)

In [ ]:
network.statistics()

In [ ]:
plot_capacity_comparison(network)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
network.buses_t.marginal_price.iloc[:72].plot(ax=ax)
ax.set_title("Marginal Price Timeseries")
ax.set_ylabel("Marginal Price ($/MWh)")
ax.set_xlabel("Time")
plt.show()


In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 5))

node = 'p67'
# Create a second y-axis for the marginal price
ax2 = ax1.twinx()

# Plot marginal price for node p64 as a dotted line on the second y-axis
network.buses_t.marginal_price[node].iloc[:72].plot(ax=ax2, linestyle='--', color='black', label='Marginal Price '+node)

# Find links connected to node p64
links_p64 = network.links[(network.links['bus0'] == node) | (network.links['bus1'] == node)]

# Plot line flows for links connected to node p64
for link in links_p64.index:
    if 'fwd' in link:
        network.links_t.p0[link].iloc[:72].plot(ax=ax1, label=f'Flow {link} (p0)')
    elif 'rev' in link:
        network.links_t.p1[link].iloc[:72].plot(ax=ax1, label=f'Flow {link} (p1)')
    # break

ax1.set_title("Marginal Price and Line Flows for Node "+node)
ax1.set_ylabel("Flow Value")
ax1.set_xlabel("Time")
ax2.set_ylabel("Marginal Price ($/MWh)")

# Combine legends from both y-axes
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='center left', bbox_to_anchor=(1.2, 0.5))

plt.tight_layout()
plt.show()

### Network Clustering Example

Lets make this a single node ("Copper plate") network via the pypsa.clustering functionality

In [ ]:
busmap = network.buses.interconnect
cols = ['Pd', 'country', 'reeds_zone']
[network.buses.drop(columns=col, inplace=True) for col in cols if col in network.buses.columns]
clustered_network = network.cluster.cluster_by_busmap(busmap)

In [ ]:
clustered_network

In [ ]:
clustered_network.optimize(snapshots=clustered_network.snapshots[0:72], solver_name="gurobi")

In [ ]:
plot_capacity_comparison(clustered_network)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
clustered_network.buses_t.marginal_price.iloc[:72].plot(ax=ax)
ax.set_title("Marginal Price Timeseries")
ax.set_ylabel("Marginal Price ($/MWh)")
ax.set_xlabel("Time")
plt.show()


In [ ]:
fig_pr, ax_pr = plot_energy_balance(clustered_network, 24 * 3)

## Now lets make this a capacity expansion problem

$$
\begin{aligned}
\min_{p, s, \bar{p}, \bar{s}, \bar{f}, f}
&\sum_{a \in A} \omega_a \Bigg[ 
\sum_{t \in T^a} \omega_{a,t} \Bigg( 
\sum_{g \in G} o_{g,a,t} \cdot p_{g,a,t} 
\Bigg) \\[0.5em]
& \quad + \sum_{g \in G} c_{g,a} \cdot \bar{p}_{g} 
+ \sum_{s \in S} c_s \cdot \bar{s}_{s} 
+ \sum_{b \in B} c_b \cdot \bar{f}_{b}
\Bigg] \\[0.5em]
\text{subject to:} \quad
&\widetilde{p}_{g,a,t}^{\text{min}} \bar{p}_{g} 
  \leq p_{g,a,t} 
  \leq \widetilde{p}_{g,a,t}^{\text{max}} \bar{p}_{g} 
  && \forall g \in G,\; a \in A,\; t \in T_a \\[0.5em]
& RD_g \leq p_{g,a,t} - p_{g,a,{t-1}} \leq RU_g 
  && \forall g \in G,\; a \in A,\; t \in T_a   \\[0.5em]
& \sum_{g \in G_z} \bar{p}_{g} \leq \bar{p}^{\text{max}}_{g,z} 
  && \forall z \in Z     \\[0.5em]
& 0 \leq SOC_{s,a,t} \leq h_s \bar{s}_{s} 
  && \forall s \in S,\; a \in A,\; t \in T_a    \\[0.5em]
& 0 \leq s_{s,a,t}^{out} \leq \bar{s}_{s} 
  && \forall s \in S,\; a \in A,\; t \in T_a     \\[0.5em]
& 0 \leq s_{s,a,t}^{in} \leq \bar{s}_{s} 
  && \forall s \in S,\; a \in A,\; t \in T_a      \\[0.5em]
& SOC_{s,a,t} = SOC_{s,a,t-1} 
  + \eta^{in} s_{s,a,t}^{in} 
  - \frac{1}{\eta^{out}} s_{s,a,t}^{out} 
  && \forall s \in S,\; a \in A,\; t \in T_a      \\[0.5em]
& \widetilde{f}_{b,a,t}^{\text{min}} \bar{f}_{b} 
  \leq f_{b,a,t} 
  \leq \widetilde{f}_{b,a,t}^{\text{max}} \bar{f}_{b} 
  && \forall b \in B,\; a \in A,\; t \in T_a     \\[0.5em]
& \sum_g p_{g,a,t,n} 
+ \sum_s s_{s,a,t,n}^{out} 
- \sum_s s_{s,a,t,n}^{in} 
- \sum_b K_{b,n} f_{b,a,t}  = D_{a,t,n}  \forall n \in N,\; a \in A,\; t \in T_a
\end{aligned}
$$

In [ ]:
network.generators.p_nom_extendable

In [ ]:
#restore the original extendable attributes
for component, attributes in extendable_attrs_backup.items():
    for attr, original_value in attributes.items():
        network.df(component)[attr] = original_value


In [ ]:
# network.loads_t.p_set /= 0.75
network.loads_t.p_set.iloc[20:24] = 50000 # set arbitrarily large load for 4 hour to show impacts of expansion
network.loads_t.p_set.iloc[:72].plot()


In [ ]:
network.links

In [ ]:
network.generators

In [ ]:
network.generators.groupby("carrier").p_nom_extendable.value_counts()

In [ ]:
network.optimize(snapshots=network.snapshots[0:72], solver_name="gurobi")

In [ ]:
plot_capacity_comparison(network)

In [ ]:
plot_energy_balance(network, 24 * 3)

#### These results should set off some alarms.... does anything look weird to you about the solution??

### Adjusting snapshot weighting

$$
\begin{aligned}
\min_{p, s, \bar{p}, \bar{s}, \bar{f}, f}
&\sum_{a \in A} \omega_a \Bigg[ 
\sum_{t \in T^a} \textcolor{red}{\omega_{a,t}} \Bigg( 
\sum_{g \in G} o_{g,a,t} \cdot p_{g,a,t} 
\Bigg) \\[0.5em]
& \quad + \sum_{g \in G} c_{g,a} \cdot \bar{p}_{g} 
+ \sum_{s \in S} c_s \cdot \bar{s}_{s} 
+ \sum_{b \in B} c_b \cdot \bar{f}_{b}
\Bigg] \\[0.5em]
\end{aligned}
$$

In [ ]:
network.snapshot_weightings.iloc[0:72] = 8760/72

In [ ]:
network.optimize(snapshots=network.snapshots[0:72], solver_name="gurobi")

In [ ]:
plot_energy_balance(network, 24 *3)

In [ ]:
plot_capacity_comparison(network)

# Custom Constraints

It is often the case that we want to add custom constraints that provide functionality beyond the core functionality in the PyPSA package. These custom constraints vary widely based on your application. But lets look more closely at the linopy model to understand how we can add these custom constraints!

Lets' implement spinning reserves taken from [pypsa examples](https://pypsa.readthedocs.io/en/latest/examples/reserve-power.html#Implementing-spinning-reserve-constraints)

In [ ]:
from IPython.display import Image, display

# Insert an image into the notebook
display(Image(filename='reserve-power-graph.webp'))


In [ ]:
clustered_network.optimize.create_model(snapshots=clustered_network.snapshots[0:72], multi_investment_periods=True)
model = clustered_network.model
model

In [ ]:
v_rp = clustered_network.model.add_variables(
    lower=0,
    coords=[clustered_network.model.variables['Generator-p'].coords['snapshot'], clustered_network.generators.index],
    name="Generator-p_reserve",
)
# model.variables['Generator-p'].coords
v_rp

In [ ]:
# set reserve requirement as 3% of the maximum load
reserve_req = clustered_network.loads_t.p_set.iloc[0:72].max().max() * 0.3 #  req = ~475

c_sum = clustered_network.model.add_constraints(
    v_rp.sum("Generator") >= reserve_req, name="GlobalConstraint-sum_of_reserves"
)
c_sum

In [ ]:
# Restrict the maximum reserve power that a generator can provide
a = 1
c_rpos = clustered_network.model.add_constraints(
    v_rp
    <= -clustered_network.model.variables["Generator-p"] + a * clustered_network.generators["p_nom"],
    name="Generator-reserve_upper_limit",
)
c_rpos

In [ ]:
# Restrict the fraction of each generator's dispatch that can be used for reserves
b = 0.5

c_rneg = clustered_network.model.add_constraints(
    v_rp <= b * clustered_network.model.variables["Generator-p"],
    name="Generator-reserve_lower_limit",
)
c_rneg

In [ ]:
model.constraints

In [ ]:
clustered_network.optimize.solve_model(solver_name="gurobi")

In [ ]:
plot_energy_balance(clustered_network, 24 * 3)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
reserve_power = clustered_network.generators_t["p_reserve"].iloc[0:72].T
grouped_reserve_power = reserve_power.groupby(clustered_network.generators.carrier).sum().T

# Get colors for each carrier
colors = [clustered_network.carriers.loc[carrier, 'color'] for carrier in grouped_reserve_power.columns]

grouped_reserve_power.plot(ax=ax, color=colors)

ax.set_title("Reserve Power Timeseries")
ax.set_ylabel("Reserve Power (MW)")
ax.set_xlabel("Time")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()


In [ ]:
clustered_network.generators_t["p_reserve"].iloc[0:72].mean().groupby(clustered_network.generators.carrier).sum().plot(kind="bar")

In [ ]:
clustered_network.generators_t["p_reserve"].iloc[0:72].sum(axis=1)